In [6]:
import numpy as np
import pandas as pd
import os
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from scipy.interpolate import UnivariateSpline
import statsmodels.api as sm
import matplotlib.pyplot as plt
import collections

pd.set_option("display.max_column", 999)


In [7]:
DATA_PATH = '../data/'

seeds = pd.read_csv(DATA_PATH + "WNCAATourneySeeds.csv")

def convert_seeds_to_sample_sub(seeds):

        sample_submission = seeds.merge(seeds, on="Season")
        sample_submission = sample_submission[["Season", "TeamID_x", "TeamID_y"]].copy()
        sample_submission = sample_submission[sample_submission["TeamID_x"] != sample_submission["TeamID_y"]]
        sample_submission["ID"] = np.where(sample_submission["TeamID_x"] < sample_submission["TeamID_y"],
                sample_submission["Season"].astype(str) + "_" + sample_submission["TeamID_x"].astype(str) + "_" + sample_submission["TeamID_y"].astype(str),
                sample_submission["Season"].astype(str) + "_" +  sample_submission["TeamID_y"].astype(str) + "_" + sample_submission["TeamID_x"].astype(str))
        sample_submission = sample_submission[["Season", "ID"]].copy()

        return sample_submission

sample_submission = convert_seeds_to_sample_sub(seeds)


In [8]:
sample_submission

,Season,ID
1,1998,1998_3163_3330
2,1998,1998_3112_3330
3,1998,1998_3301_3330
4,1998,1998_3272_3330
5,1998,1998_3330_3438
...,...,...
103450,2023,2023_3151_3385
103451,2023,2023_3151_3405
103452,2023,2023_3151_3387
103453,2023,2023_3151_3241


In [9]:
def prepare_data(df):
    dfswap = df[['Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'WLoc', 'NumOT', 
    'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 
    'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]

    dfswap.loc[df['WLoc'] == 'H', 'WLoc'] = 'A'
    dfswap.loc[df['WLoc'] == 'A', 'WLoc'] = 'H'
    df.columns.values[6] = 'location'
    dfswap.columns.values[6] = 'location'    
      
    df.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(df.columns)]
    dfswap.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(dfswap.columns)]

    output = pd.concat([df, dfswap]).reset_index(drop=True)
    output.loc[output.location=='N','location'] = '0'
    output.loc[output.location=='H','location'] = '1'
    output.loc[output.location=='A','location'] = '-1'
    output.location = output.location.astype(int)
    
    output['PointDiff'] = output['T1_Score'] - output['T2_Score']
    
    return output

def team_quality(season, regular_season_effects):
    formula = 'win~-1+T1_TeamID+T2_TeamID'
    glm = sm.GLM.from_formula(formula=formula, 
                              data=regular_season_effects.loc[regular_season_effects.Season==season,:], 
                              family=sm.families.Binomial()).fit()
    
    quality = pd.DataFrame(glm.params).reset_index()
    quality.columns = ['TeamID','quality']
    quality['Season'] = season
    #quality['quality'] = np.exp(quality['quality'])
    quality = quality.loc[quality.TeamID.str.contains('T1_')].reset_index(drop=True)
    quality['TeamID'] = quality['TeamID'].apply(lambda x: x[10:14]).astype(int)
    return quality

def cauchyobj(preds, dtrain):
    labels = dtrain.get_label()
    c = 5000 
    x =  preds-labels    
    grad = x / (x**2/c**2+1)
    hess = -c**2*(x**2-c**2)/(x**2+c**2)**2
    return grad, hess


In [10]:
def raddar_pipeline(seeds, regular_results, tourney_results, sub):

    regular_results = regular_results.copy()
    tourney_results = tourney_results.copy()

    print(regular_results.columns)
    
    regular_data = prepare_data(regular_results)
    tourney_data = prepare_data(tourney_results)

    boxscore_cols = ['T1_Score', 'T2_Score', 
        'T1_FGM', 'T1_FGA', 'T1_FGM3', 'T1_FGA3', 'T1_FTM', 'T1_FTA', 'T1_OR', 'T1_DR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_Blk', 'T1_PF', 
        'T2_FGM', 'T2_FGA', 'T2_FGM3', 'T2_FGA3', 'T2_FTM', 'T2_FTA', 'T2_OR', 'T2_DR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk', 'T2_PF', 
        'PointDiff']

    boxscore_cols = [
            'T1_FGM', 'T1_FGA', 'T1_FGM3', 'T1_FGA3', 'T1_OR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_PF', 
            'T2_FGM', 'T2_FGA', 'T2_FGM3', 'T2_FGA3', 'T2_OR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk',  
            'PointDiff']

    funcs = [np.mean]

    season_statistics = regular_data.groupby(["Season", 'T1_TeamID'])[boxscore_cols].agg(funcs).reset_index()
    season_statistics.columns = [''.join(col).strip() for col in season_statistics.columns.values]

    season_statistics_T1 = season_statistics.copy()
    season_statistics_T2 = season_statistics.copy()

    season_statistics_T1.columns = ["T1_" + x.replace("T1_","").replace("T2_","opponent_") for x in list(season_statistics_T1.columns)]
    season_statistics_T2.columns = ["T2_" + x.replace("T1_","").replace("T2_","opponent_") for x in list(season_statistics_T2.columns)]
    season_statistics_T1.columns.values[0] = "Season"
    season_statistics_T2.columns.values[0] = "Season"

    tourney_data = tourney_data[['Season', 'DayNum', 'T1_TeamID', 'T1_Score', 'T2_TeamID' ,'T2_Score']]
    tourney_data = pd.merge(tourney_data, season_statistics_T1, on = ['Season', 'T1_TeamID'], how = 'left')
    tourney_data = pd.merge(tourney_data, season_statistics_T2, on = ['Season', 'T2_TeamID'], how = 'left')

    last14days_stats_T1 = regular_data.loc[regular_data.DayNum>118].reset_index(drop=True)
    last14days_stats_T1['win'] = np.where(last14days_stats_T1['PointDiff']>0,1,0)
    last14days_stats_T1 = last14days_stats_T1.groupby(['Season','T1_TeamID'])['win'].mean().reset_index(name='T1_win_ratio_14d')

    last14days_stats_T2 = regular_data.loc[regular_data.DayNum>118].reset_index(drop=True)
    last14days_stats_T2['win'] = np.where(last14days_stats_T2['PointDiff']<0,1,0)
    last14days_stats_T2 = last14days_stats_T2.groupby(['Season','T2_TeamID'])['win'].mean().reset_index(name='T2_win_ratio_14d')

    tourney_data = pd.merge(tourney_data, last14days_stats_T1, on = ['Season', 'T1_TeamID'], how = 'left')
    tourney_data = pd.merge(tourney_data, last14days_stats_T2, on = ['Season', 'T2_TeamID'], how = 'left')

    regular_season_effects = regular_data[['Season','T1_TeamID','T2_TeamID','PointDiff']].copy()
    regular_season_effects['T1_TeamID'] = regular_season_effects['T1_TeamID'].astype(str)
    regular_season_effects['T2_TeamID'] = regular_season_effects['T2_TeamID'].astype(str)
    regular_season_effects['win'] = np.where(regular_season_effects['PointDiff']>0,1,0)
    march_madness = pd.merge(seeds[['Season','TeamID']],seeds[['Season','TeamID']],on='Season')
    march_madness.columns = ['Season', 'T1_TeamID', 'T2_TeamID']
    march_madness.T1_TeamID = march_madness.T1_TeamID.astype(str)
    march_madness.T2_TeamID = march_madness.T2_TeamID.astype(str)
    regular_season_effects = pd.merge(regular_season_effects, march_madness, on = ['Season','T1_TeamID','T2_TeamID'])
    glm_quality = pd.concat([
                         team_quality(2003, regular_season_effects),
                         team_quality(2004, regular_season_effects),
                         team_quality(2005, regular_season_effects),
                         team_quality(2006, regular_season_effects),
                         team_quality(2007, regular_season_effects),
                         team_quality(2008, regular_season_effects),
                         team_quality(2009, regular_season_effects),
                         team_quality(2010, regular_season_effects),
                         team_quality(2011, regular_season_effects),
                         team_quality(2012, regular_season_effects),
                         team_quality(2013, regular_season_effects),
                         team_quality(2014, regular_season_effects),
                         team_quality(2015, regular_season_effects),
                         team_quality(2016, regular_season_effects),
                         team_quality(2017, regular_season_effects),
                         team_quality(2018, regular_season_effects),
                         team_quality(2019, regular_season_effects),
                         ##team_quality(2020),
                         team_quality(2021, regular_season_effects),
                         team_quality(2022, regular_season_effects),
                         team_quality(2023, regular_season_effects)
                         ]).reset_index(drop=True)

                    
    glm_quality_T1 = glm_quality.copy()
    glm_quality_T2 = glm_quality.copy()
    glm_quality_T1.columns = ['T1_TeamID','T1_quality','Season']
    glm_quality_T2.columns = ['T2_TeamID','T2_quality','Season']
    
    tourney_data = pd.merge(tourney_data, glm_quality_T1, on = ['Season', 'T1_TeamID'], how = 'left')
    tourney_data = pd.merge(tourney_data, glm_quality_T2, on = ['Season', 'T2_TeamID'], how = 'left')

    seeds['seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))

    seeds_T1 = seeds[['Season','TeamID','seed']].copy()
    seeds_T2 = seeds[['Season','TeamID','seed']].copy()
    seeds_T1.columns = ['Season','T1_TeamID','T1_seed']
    seeds_T2.columns = ['Season','T2_TeamID','T2_seed']

    tourney_data = pd.merge(tourney_data, seeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
    tourney_data = pd.merge(tourney_data, seeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')

    tourney_data["Seed_diff"] = tourney_data["T1_seed"] - tourney_data["T2_seed"]

    # Build
    y = tourney_data['T1_Score'] - tourney_data['T2_Score']

    features = list(season_statistics_T1.columns[2:999]) + \
    list(season_statistics_T2.columns[2:999]) + \
    list(seeds_T1.columns[2:999]) + \
    list(seeds_T2.columns[2:999]) + \
    list(last14days_stats_T1.columns[2:999]) + \
    list(last14days_stats_T2.columns[2:999]) + \
    ["Seed_diff"] + ["T1_quality","T2_quality"]

    X = tourney_data[features].values
    dtrain = xgb.DMatrix(X, label = y)

    param = {} 
    #param['objective'] = 'reg:linear'
    param['eval_metric'] =  'mae'
    param['booster'] = 'gbtree'
    param['eta'] = 0.05 #change to ~0.02 for final run
    param['subsample'] = 0.35
    param['colsample_bytree'] = 0.7
    param['num_parallel_tree'] = 3 #recommend 10
    param['min_child_weight'] = 40
    param['gamma'] = 10
    param['max_depth'] =  3
    param['silent'] = 1

    xgb_cv = []
    repeat_cv = 3 # recommend 10

    for i in range(repeat_cv): 
        print(f"Fold repeater {i}")
        xgb_cv.append(
            xgb.cv(
            params = param,
            dtrain = dtrain,
            obj = cauchyobj,
            num_boost_round = 3000,
            folds = KFold(n_splits = 5, shuffle = True, random_state = i),
            early_stopping_rounds = 25,
            verbose_eval = 50
            )
        )
    iteration_counts = [np.argmin(x['test-mae-mean'].values) for x in xgb_cv]
    val_mae = [np.min(x['test-mae-mean'].values) for x in xgb_cv]

    oof_preds = []
    for i in range(repeat_cv):
        print(f"Fold repeater {i}")
        preds = y.copy()
        kfold = KFold(n_splits = 5, shuffle = True, random_state = i)    
        for train_index, val_index in kfold.split(X,y):
            dtrain_i = xgb.DMatrix(X[train_index], label = y[train_index])
            dval_i = xgb.DMatrix(X[val_index], label = y[val_index])  
            model = xgb.train(
                params = param,
                dtrain = dtrain_i,
                num_boost_round = iteration_counts[i],
                verbose_eval = 50
            )
            preds[val_index] = model.predict(dval_i)
        oof_preds.append(np.clip(preds,-30,30))

    val_cv = []
    spline_model = []

    for i in range(repeat_cv):
        dat = list(zip(oof_preds[i],np.where(y>0,1,0)))
        dat = sorted(dat, key = lambda x: x[0])
        datdict = {}
        for k in range(len(dat)):
            datdict[dat[k][0]]= dat[k][1]
        spline_model.append(UnivariateSpline(list(datdict.keys()), list(datdict.values())))
        spline_fit = spline_model[i](oof_preds[i])
        spline_fit = np.clip(spline_fit,0.025,0.975)
        spline_fit[(tourney_data.T1_seed==1) & (tourney_data.T2_seed==16) & (tourney_data.T1_Score > tourney_data.T2_Score)] = 1.0
        spline_fit[(tourney_data.T1_seed==2) & (tourney_data.T2_seed==15) & (tourney_data.T1_Score > tourney_data.T2_Score)] = 1.0
        spline_fit[(tourney_data.T1_seed==3) & (tourney_data.T2_seed==14) & (tourney_data.T1_Score > tourney_data.T2_Score)] = 1.0
        spline_fit[(tourney_data.T1_seed==4) & (tourney_data.T2_seed==13) & (tourney_data.T1_Score > tourney_data.T2_Score)] = 1.0
        spline_fit[(tourney_data.T1_seed==16) & (tourney_data.T2_seed==1) & (tourney_data.T1_Score < tourney_data.T2_Score)] = 0.0
        spline_fit[(tourney_data.T1_seed==15) & (tourney_data.T2_seed==2) & (tourney_data.T1_Score < tourney_data.T2_Score)] = 0.0
        spline_fit[(tourney_data.T1_seed==14) & (tourney_data.T2_seed==3) & (tourney_data.T1_Score < tourney_data.T2_Score)] = 0.0
        spline_fit[(tourney_data.T1_seed==13) & (tourney_data.T2_seed==4) & (tourney_data.T1_Score < tourney_data.T2_Score)] = 0.0
        
        val_cv.append(pd.DataFrame({"y":np.where(y>0,1,0), "pred":spline_fit, "season":tourney_data.Season}))
        print(f"adjusted logloss of cvsplit {i}: {log_loss(np.where(y>0,1,0),spline_fit)}") 
        
    val_cv = pd.concat(val_cv)
    val_cv.groupby('season').apply(lambda x: log_loss(x.y, x.pred))
        
    sub['Season'] = sub['ID'].apply(lambda x: int(x.split('_')[0]))
    sub["T1_TeamID"] = sub['ID'].apply(lambda x: int(x.split('_')[1]))
    sub["T2_TeamID"] = sub['ID'].apply(lambda x: int(x.split('_')[2]))

    sub = pd.merge(sub, season_statistics_T1, on = ['Season', 'T1_TeamID'], how = 'left')
    sub = pd.merge(sub, season_statistics_T2, on = ['Season', 'T2_TeamID'], how = 'left')

    sub = pd.merge(sub, glm_quality_T1, on = ['Season', 'T1_TeamID'], how = 'left')

    sub = pd.merge(sub, glm_quality_T2, on = ['Season', 'T2_TeamID'], how = 'left')

    sub = pd.merge(sub, seeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
    sub = pd.merge(sub, seeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')
    sub = pd.merge(sub, last14days_stats_T1, on = ['Season', 'T1_TeamID'], how = 'left')
    sub = pd.merge(sub, last14days_stats_T2, on = ['Season', 'T2_TeamID'], how = 'left')

    sub["Seed_diff"] = sub["T1_seed"] - sub["T2_seed"]

    Xsub = sub[features].values
    dtest = xgb.DMatrix(Xsub)

    sub_models = []
    for i in range(repeat_cv):
        print(f"Fold repeater {i}")
        sub_models.append(
            xgb.train(
            params = param,
            dtrain = dtrain,
            num_boost_round = int(iteration_counts[i] * 1.05),
            verbose_eval = 50
            )
        )

    sub_preds = []
    for i in range(repeat_cv):
        sub_preds.append(np.clip(spline_model[i](np.clip(sub_models[i].predict(dtest),-30,30)),0.025,0.975))
        
    sub["Pred"] = pd.DataFrame(sub_preds).mean(axis=0)
    return sub[['ID','Pred']]



In [11]:
# preds_2023 = raddar_pipeline(seeds, regular_results, tourney_results[tourney_results != 2023], sample_submission[sample_submission.Season == 2023])


In [12]:
#preds_2023.to_csv("../submissions/raddar_2023.csv")

In [13]:
import copy


In [14]:
all_raddar_preds = []

for s in range(2010, 2024):

    print(s)

    tourney_results = pd.concat([
    pd.read_csv(DATA_PATH + "MNCAATourneyDetailedResults.csv"),
    pd.read_csv(DATA_PATH + "WNCAATourneyDetailedResults.csv"),
    ], ignore_index=True)

    seeds = pd.concat([
        pd.read_csv(DATA_PATH + "MNCAATourneySeeds.csv"),
        pd.read_csv(DATA_PATH + "WNCAATourneySeeds.csv"),
    ], ignore_index=True)

    regular_results = pd.concat([
        pd.read_csv(DATA_PATH + "MRegularSeasonDetailedResults.csv"),
        pd.read_csv(DATA_PATH + "WRegularSeasonDetailedResults.csv"),
    ], ignore_index=True)

    preds = raddar_pipeline(seeds, regular_results, tourney_results[tourney_results["Season"] < int(s)].copy(), sample_submission[sample_submission.Season == s].copy())
    all_raddar_preds.append(preds)


2010
Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')


/Users/skylerdale/.virtualenvs/ncaa_model/lib/python3.8/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


Fold repeater 0
[20:04:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[20:04:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[20:04:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings

/Users/skylerdale/.virtualenvs/ncaa_model/lib/python3.8/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


Fold repeater 0
[20:05:33] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[20:05:33] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[20:05:33] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings

...

Fold repeater 0
[20:17:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[20:17:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[20:17:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings

/Users/skylerdale/.virtualenvs/ncaa_model/lib/python3.8/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


Fold repeater 0
[20:18:57] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[20:18:57] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[20:18:57] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings

/Users/skylerdale/.virtualenvs/ncaa_model/lib/python3.8/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


Fold repeater 0
[20:20:57] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[20:20:57] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[20:20:57] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings

In [ ]:
all_raddar_preds = pd.concat(all_raddar_preds, axis=0)

In [ ]:
all_raddar_preds

,ID,Pred
0,2010_1246_1452,0.583944
1,2010_1246_1307,0.625559
2,2010_1246_1458,0.627535
3,2010_1246_1396,0.671084
4,2010_1246_1266,0.731802
...,...,...
4551,2023_1224_1305,0.275712
4552,2023_1224_1433,0.289068
4553,2023_1224_1233,0.259827
4554,2023_1213_1224,0.537069


In [ ]:
all_raddar_preds.to_csv("../submissions/all_raddar_preds_women.csv")